In [ ]:
using constants

using types

using Optim

using FastAnonymous

E_lowlimit=-1.0

E_highlimit=1.0

interval=0.01

xmax=3.0

Temp=300.0

Ef=0.9

Erange=collect(E_lowlimit:interval:E_highlimit)

band1=parBand(1.0*me,0.0,0.5)

band2=parBand(-2.0*me,0.0,-0.2)

bndst=BandStruc([band1,band2])

function Numofn(band::parBand,Ef,Temp::Float64,xmax::Float64)

    if band.effMass>0.0

        integrand(E)=q*getDOS_Epoint(band,E).*(-1*fermiStatpoint(Temp,Ef,E))

        a=quadgk(integrand,-1*xmax,xmax)

    else

        integrand(E)=q*getDOS_Epoint(band,E).*(1-fermiStatpoint(Temp,Ef,E))

        a=quadgk(integrand,-1*xmax,xmax)

    end

    

    return a[1]

end

function getDOS_Epoint(band::parBand,E)

     if band.effMass<0.0 

            if -E-band.offset>=0            

            dos=real(im*(pi*8*sqrt(2).*Complex(band.effMass).^(3.0/2.0).*sqrt((-E-band.offset).*q)/h/h/h))

             else

                 dos=0.0

             end

        end

        if band.effMass>0.0 

            if E>=band.offset         

               dos=real((pi*8*sqrt(2).*Complex(band.effMass).^(3.0/2.0).*sqrt((E-band.offset)*q)/h/h/h))

             else

                dos=0.0

             end

        end

    return dos

end

function fermiStatpoint(Temp::Float64,Ef,E::Float64)

   return 1.0./(1.0+exp(q*(E-Ef)./kB./Temp))    

end

function NumofnMultiBand(bndst::BandStruc,Ef,Temp::Float64,xmax::Float64)

    totalnumofn=0.0

    for band in bndst.bands

        totalnumofn=totalnumofn+Numofn(band,Ef,Temp,xmax) 

    end

    return totalnumofn 

end

function Numofn(band::parBand,Ef,Temp::Float64,xmax::Float64)

    if band.effMass>0.0
        integrand(E)=-E-band.offset>=0 ? q*real(im*(pi*8*sqrt(2).*Complex(band.effMass).^(3.0/2.0).*sqrt((-E-band.offset).*q)/h/h/h)).*(-1*fermiStatpoint(Temp,Ef,E)):0.0
        a=quadgk(integrand,-1*xmax,xmax)

    else

        integrand(E)=q*getDOS_Epoint(band,E).*(1-fermiStatpoint(Temp,Ef,E))

        a=quadgk(integrand,-1*xmax,xmax)

    end    

    return a[1]

end

@time numofnoutput=Numofn(band1,Ef,Temp,xmax)/1e6

function FindFermiLevel(numberofn::Float64,bndst::BandStruc,Ef,Temp::Float64,xmax::Float64)

    return abs(numberofn*1e6-NumofnMultiBand(bndst,Ef,Temp,xmax))

end

numofninput=numofnoutput

equation(x)=FindFermiLevel(numofninput,bndst,x,Temp,xmax)

@time Efcalc=optimize(equation,-1.0,1.0,method=:golden_section)
